In [ ]:
from xml.etree import ElementTree as ET

def parse_XML_log(event):
    tree = ET.ElementTree(ET.fromstring(event))
    root = tree.getroot()
    ns = "{http://schemas.microsoft.com/win/2004/08/events/event}"
    data = {}
    for eventID in root.findall(".//"):
        if eventID.tag == f"{ns}System":
            for e_id in eventID.iter():
                if e_id.tag == f"{ns}System":
                    pass
                elif e_id.tag == f"{ns}Provider":
                    data["Provider"] = e_id.attrib.get('Name')
                elif e_id.tag == f"{ns}TimeCreated":
                    data["TimeCreated"] = e_id.attrib.get('SystemTime')
                elif e_id.tag == f"{ns}Correlation":
                    data["ActivityID"] = e_id.attrib.get('ActivityID')
                elif e_id.tag == f"{ns}Execution":
                    data["ProcessID"] = e_id.attrib.get('ProcessID')
                    data["ThreadID"] = e_id.attrib.get('ThreadID')
                else:
                    att = e_id.tag.replace(f"{ns}", "")
                    data[att] = e_id.text

        if eventID.tag == f"{ns}EventData":
            for attr in eventID.iter():
                if attr.tag == f'{ns}Data':
                    if attr.get('Name') is None:
                        data["Data"] = attr.text
                    else:
                        data[attr.get('Name')] = attr.text
                elif attr.tag == f'{ns}Binary':
                    data["Binary"] = attr.text

    return data

In [ ]:
from json import loads
from kafka import KafkaConsumer
from pymongo import MongoClient


# generating the Kafka Consumer
my_consumer = KafkaConsumer(
    'users',
    bootstrap_servers=['192.168.0.2:9092'],
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id='my-group',
    value_deserializer=lambda x: loads(x.decode('utf-8'))
)


try:
   my_client = MongoClient('127.0.0.1',27017)
   my_collection = my_client.my_application.users
   print("Connected successfully!")
except:  
   print("Could not connect to MongoDB")


for message in my_consumer:
   message = message.value
   log = parse_XML_log(message)
   try:
      my_collection.insert_one(log)
      print("Data inserted successfull!")
   except:
      print("Could not insert into MongoDB!")
   
